In [1]:
import numpy as np 
import pandas as pd 
import re
import nltk
import json
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
from collections import Counter
from string import punctuation, ascii_lowercase
from gensim.models import Word2Vec
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout,SpatialDropout1D, Bidirectional
from keras.layers import Conv1D,MaxPooling1D,Dropout,Activation,Flatten,GlobalMaxPooling1D,Reshape
from keras.models import Model,load_model
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
filename = "C:\\Kshitij\\DEV\\Datasets\\spider\\train_spider.json"
# data_Set = open("C:\\Kshitij\\DEV\\Datasets\\spider\\train_others.json")
queries =[]
questions = []
with open(filename, 'r') as f:
    datastore = json.load(f)

for i in range(len(datastore)):
	queries.append(datastore[i]['query'])
	questions.append(datastore[i]['question'])

first = "WHERE"
second = "JOIN"

first_cnt = 0 
second_cnt = 0 
zero_cnt = 0

train_data = []
label = [-1]*5349

j = -1
for i in range(len(queries)):
	if second not in queries[i] and first not in queries[i]:
		train_data.append(queries[i])
		j += 1
		label[j] = [0,0,1]
	if second in queries[i] and first not in queries[i]:
		train_data.append(queries[i])
		j += 1
		label[j] = [0,1,0]
	if second not in queries[i] and first in queries[i]:
		train_data.append(queries[i])
		j += 1
		label[j] = [1,0,0]

In [3]:

filename = "C:\\Kshitij\\DEV\\Datasets\\spider\\train_others.json"
# data_Set = open("C:\\Kshitij\\DEV\\Datasets\\spider\\train_others.json")
pred_queries =[]
with open(filename, 'r') as f:
    datastore = json.load(f)

for i in range(len(datastore)):
	pred_queries.append(datastore[i]['query'])
# print(pred_queries[:5])
print(len(pred_queries))
label_pred = []
test_data = []
j = -1
for i in range(len(pred_queries)):
	if second not in queries[i] and first not in queries[i]:
		test_data.append(queries[i])
		j += 1
		label_pred.append([0,0,1])
	if second in queries[i] and first not in queries[i]:
		test_data.append(queries[i])
		j += 1
		label_pred.append([0,1,0])
	if second not in queries[i] and first in queries[i]:
		test_data.append(queries[i])
		j += 1
		label_pred.append([1,0,0])
# test_data.append("SELECT * FROM students")
# test_data.append("SELECT * FROM students WHERE id = 1")
# test_data.append("SELECT Orders.OrderID, Customers.CustomerName, Orders.OrderDate FROM Orders INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID")
# label_pred.append([0,0,1])
# label_pred.append([0,1,0])
# label_pred.append([1,0,0])
print(len(test_data))

1659
1332


In [4]:
model = load_model("../queries_cat1.h5")
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 55)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 55, 100)           219400    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 55, 100)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 53, 128)           38528     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 17, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 17, 128)           49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 5, 128)            0   

In [5]:
stop_words = set(nltk.corpus.stopwords.words("english"))
tokenizer = WordPunctTokenizer() 
vocab = Counter()

def text_to_wordlist(text, lower=False):
    words = ""
    text = tokenizer.tokenize(text)
    text = [t.lower() for t in text]
    vocab.update(text)
    return text

def process_comments(list_sentences, lower=False):
    comments = []
    for text in list_sentences:
        txt = text_to_wordlist(text, lower=lower)
        comments.append(txt)
    return comments

In [6]:
list_sentences_pred = test_data
queries_test = process_comments(list_sentences_pred, lower=True)
print(queries_test[:3])

[['select', 'count', '(*)', 'from', 'head', 'where', 'age', '>', '56'], ['select', 'name', ',', 'born_state', ',', 'age', 'from', 'head', 'order', 'by', 'age'], ['select', 'creation', ',', 'name', ',', 'budget_in_billions', 'from', 'department']]


In [7]:
pickle_in = open("word_index.pickle","rb")
word_index = pickle.load(pickle_in)
MAX_SEQUENCE_LENGTH = 55

In [8]:
sequences_pred = [[word_index.get(t, 0) for t in query] for query in queries_test]
data_pred = pad_sequences(sequences_pred, maxlen=MAX_SEQUENCE_LENGTH,padding="pre")
label_pred = np.array(label_pred)
print(sequences_pred[:10])
print(len(test_data))
print(data_pred.shape)
print(label_pred.shape)

[[1, 10, 12, 2, 692, 11, 32, 26, 1999], [1, 22, 8, 1071, 8, 32, 2, 692, 16, 6, 32], [1, 1353, 8, 22, 8, 1354, 2, 73], [1, 45, 14, 1354, 15, 8, 50, 14, 1354, 15, 2, 73], [1, 28, 14, 2000, 15, 2, 73, 11, 1426, 103, 136, 31, 536], [1, 22, 2, 692, 11, 1071, 72, 24, 1355, 24], [1, 1071, 2, 692, 13, 6, 1071, 29, 10, 12, 44, 56], [1, 1353, 2, 73, 13, 6, 1353, 16, 6, 10, 12, 23, 21, 20], [1, 10, 14, 27, 2001, 15, 2, 1356], [1, 10, 12, 2, 73, 11, 66, 41, 35, 14, 1, 66, 2, 1356, 615]]
1332
(1332, 55)
(1332, 3)


In [20]:
print(queries_test)

[['select', 'count', '(*)', 'from', 'head', 'where', 'age', '>', '56'], ['select', 'name', ',', 'born_state', ',', 'age', 'from', 'head', 'order', 'by', 'age'], ['select', 'creation', ',', 'name', ',', 'budget_in_billions', 'from', 'department'], ['select', 'max', '(', 'budget_in_billions', ')', ',', 'min', '(', 'budget_in_billions', ')', 'from', 'department'], ['select', 'avg', '(', 'num_employees', ')', 'from', 'department', 'where', 'ranking', 'between', '10', 'and', '15'], ['select', 'name', 'from', 'head', 'where', 'born_state', '!=', "'", 'california', "'"], ['select', 'born_state', 'from', 'head', 'group', 'by', 'born_state', 'having', 'count', '(*)', '>=', '3'], ['select', 'creation', 'from', 'department', 'group', 'by', 'creation', 'order', 'by', 'count', '(*)', 'desc', 'limit', '1'], ['select', 'count', '(', 'distinct', 'temporary_acting', ')', 'from', 'management'], ['select', 'count', '(*)', 'from', 'department', 'where', 'department_id', 'not', 'in', '(', 'select', 'depart

In [9]:
pred_Comm = model.predict(data_pred)
print(pred_Comm)
print([np.argmax(i)for i in pred_Comm])
print(model.evaluate(data_pred,label_pred))
print(model.metrics_names)

[[9.9611014e-01 1.3190230e-05 3.8766451e-03]
 [7.4872246e-04 3.0216550e-05 9.9922109e-01]
 [5.3878251e-02 2.8400670e-04 9.4583768e-01]
 ...
 [9.9996603e-01 1.5196545e-06 3.2414024e-05]
 [9.9999237e-01 1.1022028e-08 7.5841522e-06]
 [9.9999237e-01 1.1022028e-08 7.5841522e-06]]
[0, 2, 2, 2, 0, 0, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 1, 1, 0, 0, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 1, 1, 2, 2, 2, 0, 2, 2, 0, 2, 2, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 

In [27]:
def categorize(x):
    query = []
    query.append(x)
    query = process_comments(query,lower=True)
    sequences_pred1 = [[word_index.get(t, 0) for t in q] for q in query]
    data_pred1 = pad_sequences(sequences_pred1, maxlen=MAX_SEQUENCE_LENGTH,padding="pre")
    pred_Comm = model.predict(data_pred1)
    print(sequences_pred1)
    print(data_pred1)
    print(pred_Comm)
    print([np.argmax(i)for i in pred_Comm])

In [28]:
categorize('SELECT * FROM STUDENTS')

[[1, 122, 2, 155]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1 122   2
  155]]
[[2.9930787e-03 2.8397000e-04 9.9672300e-01]]
[2]
